In [3]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [13]:

loader = PyMuPDFLoader("data/oraclePatch.pdf")
docs = loader.load()
print(f"Number of Document: {len(docs)}")

Number of Document: 10


In [14]:
print (docs[9].page_content)

Oracle Patch
10
The script 
@utlrp.sql  is an Oracle-supplied SQL script used to recompile invalid objects in the 
database. Here's a detailed explanation of its purpose and usage:
cd %ORACLE_HOME%/rdbms/admin
sqlplus /nolog
CONNECT / AS SYSDBA
@utlrp.sql
Conclusion
Following these steps ensures that your Oracle Database is patched correctly and 
efficiently in a Windows environment. Always verify each step and consult the official 
Oracle documentation for any specific instructions or additional details.



### Check the metadata

In [15]:
docs[9].__dict__

{'id': None,
 'metadata': {'source': 'data/oraclePatch.pdf',
  'file_path': 'data/oraclePatch.pdf',
  'page': 9,
  'total_pages': 10,
  'format': 'PDF 1.4',
  'title': '',
  'author': '',
  'subject': '',
  'keywords': '',
  'creator': 'Chromium',
  'producer': 'Skia/PDF m119',
  'creationDate': "D:20240612210933+00'00'",
  'modDate': "D:20240612210933+00'00'",
  'trapped': ''},
 'page_content': "Oracle Patch\n10\nThe script \n@utlrp.sql  is an Oracle-supplied SQL script used to recompile invalid objects in the \ndatabase. Here's a detailed explanation of its purpose and usage:\ncd %ORACLE_HOME%/rdbms/admin\nsqlplus /nolog\nCONNECT / AS SYSDBA\n@utlrp.sql\nConclusion\nFollowing these steps ensures that your Oracle Database is patched correctly and \nefficiently in a Windows environment. Always verify each step and consult the official \nOracle documentation for any specific instructions or additional details.\n",
 'type': 'Document'}

In [17]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"chunck: {len(split_documents)}")

chunck: 23


text-embedding-3-large and text-embedding-3-small

In [19]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [20]:
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [23]:
for doc in vectorstore.similarity_search("services"):
    print(doc.page_content)

Oracle Patch
7
7. Stopping Services
Before applying the patch, stop the following services to prevent any conflicts or issues:
1. Clear PERL5LIB environment variable:
set PERL5LIB=
The command set PERL5LIB=  is used to clear the PERL5LIB  environment variable. This 
variable is typically used to specify the path for Perl libraries.
In the context of the Oracle patch process, setting PERL5LIB  to an empty value ensures
Oracle Patch
2
Oracle Patch
4
2. Login as svcDBamnd13 (service account: This is DEV 
environment) to Amanda DB Server
1. Find the password from secret server.
3. Setting Up Environment Variables
1. Set the Oracle SID, Oracle Base, and Oracle Home:
Open a command prompt as admin and enter the following commands to set up the 
necessary environment variables:
SET ORACLE_SID=AMND13
SET ORACLE_BASE=E:\app\oracle
SET ORACLE_HOME=E:\app\oracle\product\19.3.0\dbhome_1
Oracle Patch
8
1. Start the services:
net start msdtc
net start OracleServiceAMND13
net start OracleVssWriterAMN

In [25]:
retriever = vectorstore.as_retriever()

In [26]:
retriever.invoke("How often should I have to patch my Oracle database?")

[Document(metadata={'source': 'data/oraclePatch.pdf', 'file_path': 'data/oraclePatch.pdf', 'page': 0, 'total_pages': 10, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Chromium', 'producer': 'Skia/PDF m119', 'creationDate': "D:20240612210933+00'00'", 'modDate': "D:20240612210933+00'00'", 'trapped': ''}, page_content='Oracle Patch\n1\nOracle Patch\nOracle Database Patching Guide for Windows \nEnvironment\nhttps://www.youtube.com/watch?v=ODyzpwz115M\nhttps://www.youtube.com/watch?v=gT0Gn1wxpuU\nThis guide details the process for patching an Oracle Database in a Windows \nenvironment. It includes setting up environment variables, checking and revoking DBA \nroles, preparing and applying the patch, and post-patch steps.\nAll the detailed information is in'),
 Document(metadata={'source': 'data/oraclePatch.pdf', 'file_path': 'data/oraclePatch.pdf', 'page': 7, 'total_pages': 10, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywor

In [29]:
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 


#Context: 
{context}

#Question:
{question}

#Answer:"""
)

In [30]:
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [31]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
question = "show post patch steps"
response = chain.invoke(question)
print(response)

In [37]:
# Step 1: Load Documents
loader = PyMuPDFLoader("data/OraclePatch.pdf")
docs = loader.load()

# Step 2: Split Documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

# Step 3: Create Embeddings
embeddings = OpenAIEmbeddings()

# Step 4: Create DB and Save
# Create the vector store
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

# Step 5: Create Retriever
# Create a retriever to search and generate information from the documents
retriever = vectorstore.as_retriever()

# Step 6: Create Prompt
# Create the prompt
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 


#Context: 
{context}

#Question:
{question}

#Answer:"""
)

# Step 7: Create Language Model (LLM)
# Create the model (LLM)
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Step 8: Create Chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [40]:
question = "Who is the CEO of Oracle?"
response = chain.invoke(question)
print(response)

I don't know.
